In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import BaggingClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Final players df.csv')

Approach 4, with additional new columns metrics - "dpm",  "damagemitigatedperminute", & "vspm"

these columns then calculated for 10th min and 15th min by applying the growth rate logic.

#Data Preprocessing

In [ ]:
def calculate_growth_rate(value_per_minute, total_time=30):
    total_value_at_time = value_per_minute * total_time
    return (total_value_at_time / value_per_minute) ** (1 / total_time) - 1

In [ ]:
def calculate_value_at_time(value_per_minute, minute, growth_rate):
    return value_per_minute * (1 + growth_rate) ** minute

In [ ]:
df['damage_at_15'] = df['dpm'].apply(lambda dpm: calculate_value_at_time(dpm, 15, calculate_growth_rate(dpm)))
df['damage_taken_at_15'] = df['damagetakenperminute'].apply(lambda dmg_taken: calculate_value_at_time(dmg_taken, 15, calculate_growth_rate(dmg_taken)))
df['damage_mitigated_at_15'] = df['damagemitigatedperminute'].apply(lambda dmg_mitigated: calculate_value_at_time(dmg_mitigated, 15, calculate_growth_rate(dmg_mitigated)))
df['vision_score_at_15'] = df['vspm'].apply(lambda vspm: calculate_value_at_time(vspm, 15, calculate_growth_rate(vspm)))

In [ ]:
columns = [
    'damage_at_15','damage_taken_at_15','damage_mitigated_at_15','vision_score_at_15','goldat15',
    'xpat15','csat15','opp_goldat15','opp_xpat15','opp_csat15','killsat15', 'assistsat15', 'deathsat15',
    'opp_killsat15','opp_assistsat15','opp_deathsat15',
    'result'
]

In [ ]:
columns_df = df[columns]

In [ ]:
columns_df.isna().sum()

,0
damage_at_15,0
damage_taken_at_15,0
damage_mitigated_at_15,0
vision_score_at_15,0
goldat15,0
xpat15,0
csat15,0
opp_goldat15,0
opp_xpat15,0
opp_csat15,0


In [ ]:
X = columns_df.drop(columns =['result'])
y = columns_df['result']

In [ ]:
X.head()

,damage_at_15,damage_taken_at_15,damage_mitigated_at_15,vision_score_at_15,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,3503.182540,4220.069209,3730.026766,5.816266,5017.0,7646.0,107.0,4293.0,6875.0,111.0,2.0,0.0,0.0,0.0,0.0,2.0
1,1873.647134,4217.887630,2225.546827,15.086470,5293.0,5306.0,93.0,4955.0,5309.0,99.0,2.0,1.0,0.0,1.0,0.0,0.0
2,3129.107203,3746.567440,3279.428016,6.361798,5333.0,7356.0,142.0,4923.0,7388.0,132.0,0.0,1.0,1.0,0.0,1.0,0.0
3,6279.300082,2508.739107,1426.684166,5.998110,5651.0,5713.0,132.0,4477.0,4479.0,119.0,1.0,2.0,0.0,0.0,0.0,2.0
4,991.353729,1376.879754,842.668964,13.450971,3685.0,3598.0,2.0,3155.0,3645.0,5.0,0.0,3.0,0.0,0.0,0.0,1.0


#Method

Splitting the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=23)

Gridsearch

In [ ]:
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'metric': ['euclidean', 'manhattan', 'chebyshev']
}

In [ ]:
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3,4,5,6]
    }

In [ ]:
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

In [ ]:
param_grid_rf = {
    'n_estimators': [10, 100, 200],
    'criterion': ['gini', 'entropy']

}

In [ ]:
param_grid_nb = {
    'var_smoothing': [1e-08, 1e-07, 1e-06, 1e-05]
}

In [ ]:
param_grid_gb = {
    'n_estimators': [10, 100, 200],

    'max_depth': [3, 4, 5, 6]
}

In [ ]:
param_grid_xgb = {
    'n_estimators': [10, 100, 200],

    'max_depth': [3, 4, 5, 6]
}

In [ ]:
param_grid_lgb = {
    'n_estimators': [10, 100, 200],

    'max_depth': [3,4,5,6]
}

In [ ]:
param_grid_bagging = {
    'n_estimators': [10, 100, 200],
    'max_features': [0.5, 0.7, 1.0]
}

Defining the models


In [ ]:
models = {
    'KNN': (KNeighborsClassifier(), param_grid_knn),
    'Decision Tree': (DecisionTreeClassifier(), param_grid_dt),
    'Logistic Regression': (LogisticRegression(max_iter=1000), param_grid_lr),
    'Random Forest': (RandomForestClassifier(), param_grid_rf)

}

In [ ]:
models2 = {

    'Naive Bayes': (GaussianNB(), param_grid_nb),
    'Gradient Boosting': (GradientBoostingClassifier(), param_grid_gb),
    'XGBoost': (XGBClassifier(), param_grid_xgb)

}

In [ ]:
models3 = {

    'LightGBM': (LGBMClassifier(), param_grid_lgb)

}

In [ ]:
models4 = {

    'Bagging': (BaggingClassifier(), param_grid_bagging)

}

using pipeling to scale the models

In [ ]:
for model_name, (model, param_grid) in models.items():


    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('model', model)
    ])


    adjusted_param_grid = {f'model__{key}': value for key, value in param_grid.items()}


    grid_search = GridSearchCV(estimator=pipeline, param_grid=adjusted_param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f'{model_name} - Best Params: {best_params}, Best Score: {best_score}')


    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy for {model_name}: {test_accuracy}')

KNN - Best Params: {'model__metric': 'manhattan', 'model__n_neighbors': 10}, Best Score: 0.6681170633955403
Test Accuracy for KNN: 0.6691353453064247
Decision Tree - Best Params: {'model__criterion': 'gini', 'model__max_depth': 6}, Best Score: 0.6550986629685335
Test Accuracy for Decision Tree: 0.6631642288465099
Logistic Regression - Best Params: {'model__C': 0.1, 'model__solver': 'liblinear'}, Best Score: 0.6963409865497441
Test Accuracy for Logistic Regression: 0.701397889279763
Random Forest - Best Params: {'model__criterion': 'gini', 'model__n_estimators': 200}, Best Score: 0.7106902012367649
Test Accuracy for Random Forest: 0.7202832808739122


KNN - Best Params: {'model__metric': 'manhattan', 'model__n_neighbors': 10}, Best Score: 0.6681170633955403

Test Accuracy for KNN: 0.6691353453064247

Decision Tree - Best Params: {'model__criterion': 'gini', 'model__max_depth': 6}, Best Score: 0.6550986629685335

Test Accuracy for Decision Tree: 0.6631642288465099

Logistic Regression - Best Params: {'model__C': 0.1, 'model__solver': 'liblinear'}, Best Score: 0.6963409865497441

Test Accuracy for Logistic Regression: 0.701397889279763

Random Forest - Best Params: {'model__criterion': 'gini', 'model__n_estimators': 200}, Best Score: 0.7106902012367649

Test Accuracy for Random Forest: 0.7202832808739122

In [ ]:
for model_name, (model, param_grid) in models2.items():


    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('model', model)
    ])


    adjusted_param_grid = {f'model__{key}': value for key, value in param_grid.items()}


    grid_search = GridSearchCV(estimator=pipeline, param_grid=adjusted_param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f'{model_name} - Best Params: {best_params}, Best Score: {best_score}')


    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy for {model_name}: {test_accuracy}')

Naive Bayes - Best Params: {'model__var_smoothing': 1e-08}, Best Score: 0.6747940598911006
Test Accuracy for Naive Bayes: 0.6763099426032216
Gradient Boosting - Best Params: {'model__max_depth': 4, 'model__n_estimators': 200}, Best Score: 0.7197047193845869
Test Accuracy for Gradient Boosting: 0.7263006850583226
XGBoost - Best Params: {'model__max_depth': 3, 'model__n_estimators': 200}, Best Score: 0.7190566723607065
Test Accuracy for XGBoost: 0.7242177374560267


Naive Bayes - Best Params: {'model__var_smoothing': 1e-08}, Best Score: 0.6747940598911006

Test Accuracy for Naive Bayes: 0.6763099426032216

Gradient Boosting - Best Params: {'model__max_depth': 4, 'model__n_estimators': 200}, Best Score: 0.7197047193845869

Test Accuracy for Gradient Boosting: 0.7263006850583226

XGBoost - Best Params: {'model__max_depth': 3, 'model__n_estimators': 200}, Best Score: 0.7190566723607065

Test Accuracy for XGBoost: 0.7242177374560267

In [ ]:
for model_name, (model, param_grid) in models3.items():


    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('model', model)
    ])


    adjusted_param_grid = {f'model__{key}': value for key, value in param_grid.items()}


    grid_search = GridSearchCV(estimator=pipeline, param_grid=adjusted_param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)


    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f'{model_name} - Best Params: {best_params}, Best Score: {best_score}')


    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy for {model_name}: {test_accuracy}')

[LightGBM] [Info] Number of positive: 34563, number of negative: 34569
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2463
[LightGBM] [Info] Number of data points in the train set: 69132, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499957 -> initscore=-0.000174
[LightGBM] [Info] Start training from score -0.000174
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

LightGBM - Best Params: {'model__max_depth': 5, 'model__n_estimators': 200}, Best Score: 0.7207924928283527

Test Accuracy for LightGBM: 0.7273190149972227

In [ ]:
for model_name, (model, param_grid) in models4.items():

    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('model', BaggingClassifier(n_jobs=1))
    ])

    adjusted_param_grid = {f'model__{key}': value for key, value in param_grid.items()}

    grid_search = GridSearchCV(estimator=pipeline, param_grid=adjusted_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    print(f'{model_name} - Best Params: {best_params}, Best Score: {best_score}')

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f'Test Accuracy for {model_name}: {test_accuracy}')

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Bagging - Best Params: {'model__max_features': 1.0, 'model__n_estimators': 200}, Best Score: 0.7089775530192263
Test Accuracy for Bagging: 0.7178300314756526


Bagging - Best Params: {'model__max_features': 1.0, 'model__n_estimators': 200}, Best Score: 0.7089775530192263

Test Accuracy for Bagging: 0.7178300314756526

#Result

KNN - Best Params: {'model__metric': 'manhattan', 'model__n_neighbors': 10}, Best Score: 0.6681170633955403

Test Accuracy for KNN: 0.6691353453064247

Decision Tree - Best Params: {'model__criterion': 'gini', 'model__max_depth': 6}, Best Score: 0.6550986629685335

Test Accuracy for Decision Tree: 0.6631642288465099

Logistic Regression - Best Params: {'model__C': 0.1, 'model__solver': 'liblinear'}, Best Score: 0.6963409865497441

Test Accuracy for Logistic Regression: 0.701397889279763

Random Forest - Best Params: {'model__criterion': 'gini', 'model__n_estimators': 200}, Best Score: 0.7106902012367649

Test Accuracy for Random Forest: 0.7202832808739122

Naive Bayes - Best Params: {'model__var_smoothing': 1e-08}, Best Score: 0.6747940598911006

Test Accuracy for Naive Bayes: 0.6763099426032216

Gradient Boosting - Best Params: {'model__max_depth': 4, 'model__n_estimators': 200}, Best Score: 0.7197047193845869

Test Accuracy for Gradient Boosting: 0.7263006850583226

XGBoost - Best Params: {'model__max_depth': 3, 'model__n_estimators': 200}, Best Score: 0.7190566723607065

Test Accuracy for XGBoost: 0.7242177374560267

LightGBM - Best Params: {'model__max_depth': 5, 'model__n_estimators': 200}, Best Score: 0.7207924928283527

Test Accuracy for LightGBM: 0.7273190149972227

Bagging - Best Params: {'model__max_features': 1.0, 'model__n_estimators': 200}, Best Score: 0.7089775530192263

Test Accuracy for Bagging: 0.7178300314756526
